In [1]:
import os
import dipy as dp
import nibabel as nib
import numpy as np
from dipy.io import read_bvals_bvecs
from dipy.core.gradients import gradient_table, reorient_bvecs, GradientTable
from bonndit import ShoreModel, ShoreFit
from bonndit.michi import fields, dwmri

DATA_DIR = "/home/olivier/Devel/test/samuel/in4michi/"

# Load fractional anisotropy
dti_fa = nib.load(os.path.join(DATA_DIR,"dti_FA.nii.gz"))

# Load DTI mask
dti_mask = nib.load(os.path.join(DATA_DIR,"mask.nii.gz"))

# Load and adjust tissue segmentation masks
csf_mask = nib.load(os.path.join(DATA_DIR,"fast_pve_0.nii.gz"))
gm_mask = nib.load(os.path.join(DATA_DIR,"fast_pve_1.nii.gz"))
wm_mask = nib.load(os.path.join(DATA_DIR,"fast_pve_2.nii.gz"))

dti_vecs = nib.load(os.path.join(DATA_DIR,"dti_V1.nii.gz"))

data = nib.load(os.path.join(DATA_DIR,"data.nii.gz")).get_data()

bvals, bvecs = read_bvals_bvecs(os.path.join(DATA_DIR,"bvals"), os.path.join(DATA_DIR,"bvecs"))
gtab = gradient_table(bvals, bvecs)

/home/olivier/Devel/venvs/bonndit/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Load fractional anisotropy
# dti_fa = nib.load(os.path.join(DATA_DIR, "dti_FA.nii.gz"))
dti_fa, meta = fields.load_scalar(os.path.join(DATA_DIR, "dti_FA.nii.gz"))

# Load DTI mask
# dti_mask = nib.load(os.path.join(DATA_DIR, "mask.nii.gz"))
dti_mask, _ = fields.load_scalar(os.path.join(DATA_DIR, "mask.nii.gz"))

# Load and adjust tissue segmentation masks
# csf_mask = nib.load(os.path.join(DATA_DIR, "fast_pve_0.nii.gz"))
csf_mask, _ = fields.load_scalar(os.path.join(DATA_DIR, "fast_pve_0.nii.gz"))
# gm_mask = nib.load(os.path.join(DATA_DIR, "fast_pve_1.nii.gz"))
gm_mask, _ = fields.load_scalar(os.path.join(DATA_DIR, "fast_pve_1.nii.gz"))
# wm_mask = nib.load(os.path.join(DATA_DIR, "fast_pve_2.nii.gz"))
wm_mask, _ = fields.load_scalar(os.path.join(DATA_DIR, "fast_pve_2.nii.gz"))

# dti_vecs = nib.load(os.path.join(DATA_DIR, "dti_V1.nii.gz"))
dti_vecs, _ = fields.load_vector(os.path.join(DATA_DIR, "dti_V1.nii.gz"))

# data = nib.load(os.path.join(DATA_DIR, "data.nii.gz"))

# bvals, bvecs = read_bvals_bvecs(os.path.join(DATA_DIR, "bvals"),
#                                os.path.join(DATA_DIR, "bvecs"))
# gtab = gradient_table(bvals, bvecs)
data, gtabm, meta = dwmri.load(os.path.join(DATA_DIR, "data.nii.gz"))

gtab = gradient_table(gtabm.bvals, gtabm.bvecs)

In [3]:
model = ShoreModel(gtab)
fit = model.fit(data, wm_mask, gm_mask, csf_mask, dti_mask, 
                    dti_fa, dti_vecs, verbose=True)

WM response: 100%|██████████| 460800/460800 [00:33<00:00, 13756.82it/s] 


In [4]:
fit.old_save('/home/olivier/Devel/test/samuel/in4michi/results/bonndit_response')
#fit.save("/home/olivier/Devel/test/samuel/in4michi/results/"+"fit.pkl")

In [5]:
fit = ShoreFit.old_load("/home/olivier/Devel/test/samuel/in4michi/results/"+"response_bonndit.npz")

In [6]:
#out, wmout, gmout, csfout, mask, meta = fit.fodf(os.path.join(DATA_DIR,"data.nii.gz"), verbose=True)

In [7]:
#img = nib.Nifti1Image(out, data.affine)
#nib.save(img, '/home/olivier/Devel/test/samuel/in4michi/bonndit_result/odf.nii.gz')